In [1]:
import csv
import sys
import re
import json
import time
import operator
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from scipy.sparse import csr_matrix,coo_matrix,csc_matrix
import pandas as pd
from datetime import datetime
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from operator import itemgetter
import scipy as sp
import scipy.sparse

In [ ]:
#Install sparse dot topn

#!pip3 install git+https://github.com/ing-bank/sparse_dot_topn.git
if sys.version_info[0] >= 3:
    from sparse_dot_topn import sparse_dot_topn as ct
    from sparse_dot_topn import sparse_dot_topn_threaded as ct_thread
else:
    import sparse_dot_topn as ct
    import sparse_dot_topn_threaded as ct_thread

In [2]:
def timeit(method):
    """
    Standard Python decorator that measures the execution time of a method;
    """
    def timed(*args, **kw):
        start = time.time()
        result = method(*args, **kw)
        end = time.time()
        
        #print(f"{method.__name__}:  {(end - start):.2f} s")
        return result
    return timed


def read_file(path: str, set_record_id_as_index: bool=False) -> pd.DataFrame:
    return pd.read_csv(path, dtype=str, escapechar="\\", index_col="record_id" if set_record_id_as_index else None)



def recall_at_k(resultTable : pd.DataFrame, trainingData: pd.DataFrame, testingData: pd.DataFrame) -> dict:
    """
    Given a list of K predictions for each query, first retrieve the correct ID from the test data,
    then look in the training data the percentage of records that have been successfully identified.
    
    For example, given query "1234-M", first retrieve the correct ID "1234" from the test data,
    then obtain from the training data all records that refer to "1234", 
    and finally look how many of them we have found;
    """
    
    # Obtain all the predictions for each record in the test set;
    perQueryRecords = resultTable.groupby("queried_record_id")
    
    # Group training records by their LinkedID truth value;
    groupedTrainingRecords = trainingData.groupby("linked_id")

    totalRecall = 0.0

    allRecords = dict()
    
    start = time.time()
    for i, (queriedRecordID, group) in enumerate(perQueryRecords):
        #if i % 1000 == 0 and i > 0:
            #print(f"processed {i}/{len(perQueryRecords)} records, {100 * i / len(perQueryRecords):.2f}%")
            #print(f"\tcurrent recall: {(totalRecall / i):.2f}")
            #print(f"\ttime elapsed: {(time.time() - start):.2f} s")
        
        try:
            queriedLinkedID = testingData.loc[queriedRecordID, "linked_id"]
        except IndexError:
            raise IndexError("ID {queriedRecordID} not found in testing data!")
        
        try:
            allRelevantRecords = set(groupedTrainingRecords.get_group(queriedLinkedID).index.values)
        except KeyError:
            allRelevantRecords = set()
        setPredictedRecords = set(group["predicted_record_id"])
        selectedRelevantRecords = setPredictedRecords.intersection(allRelevantRecords)
        recall = 1
        if (len(allRelevantRecords) > 0):
            recall = len(selectedRelevantRecords) / len(allRelevantRecords)

        totalRecall += recall
        allRecords[queriedRecordID] = [queriedRecordID, recall, len(selectedRelevantRecords), len(allRelevantRecords)]
    
    # Store the results in a summary table;
    result_table =  pd.DataFrame.from_dict(
                        allRecords,
                        orient='index',
                        columns=["QueriedRecordID", "Recall@K", "SelectedRecords", "AllRelevantRecords"]
                    )
    # Compute the filtered recall, which considers only queries with at least one relevant record in the training data;
    queries_with_relevant_records = result_table[result_table["AllRelevantRecords"] > 0]
    filtered_recall = np.mean(queries_with_relevant_records["SelectedRecords"] / queries_with_relevant_records["AllRelevantRecords"])

    return {
            "AverageRecall" : totalRecall / len(perQueryRecords),
            "AverageFilteredRecall": filtered_recall,
            "perQueryResult" : result_table
            }
    
def precision_at_k(resultTable : pd.DataFrame, trainingData: pd.DataFrame, testingData: pd.DataFrame) -> dict:
    """
    Given a list of K predictions for each query, first retrieve the correct ID from the test data,
    then look in the training data the percentage of records that are actually relevant;
    
    For example, given query "1234-M", first retrieve the correct ID "1234" from the test data,
    then obtain from the training data all records that refer to "1234", 
    and finally look how many of the records we have found are actually referring to "1234"
    """
    
    # Obtain all the predictions for each record in the test set;
    perQueryRecords = resultTable.groupby("queried_record_id")
    
    # Group training records by their LinkedID truth value;
    groupedTrainingRecords = trainingData.groupby("linked_id")

    totalPrecision = 0.0
    numberOfPredictionsForRelevantRecords = 0

    allRecords = dict()
    
    start = time.time()
    for i, (queriedRecordID, group) in enumerate(perQueryRecords):
        #if i % 1000 == 0 and i > 0:
            #print(f"processed {i}/{len(perQueryRecords)} records, {100 * i / len(perQueryRecords):.2f}%")
            #print(f"\tcurrent precision: {(totalPrecision / i):.2f}")
            #print(f"\ttime elapsed: {(time.time() - start):.2f} s")
        
        try:
            queriedLinkedID = testingData.loc[queriedRecordID, "linked_id"]
        except IndexError:
            raise IndexError("ID {queriedRecordID} not found in testing data!")
        
        try:
            allRelevantRecords = set(groupedTrainingRecords.get_group(queriedLinkedID).index.values)
        except KeyError:
            allRelevantRecords = set()
        setPredictedRecords = set(group["predicted_record_id"])
        selectedRelevantRecords = setPredictedRecords.intersection(allRelevantRecords)
        precision = 1
        if (len(allRelevantRecords) > 0):
            precision = len(selectedRelevantRecords) / len(setPredictedRecords)
            numberOfPredictionsForRelevantRecords += len(setPredictedRecords)

        totalPrecision += precision
        allRecords[queriedRecordID] = [queriedRecordID, precision, len(selectedRelevantRecords), len(allRelevantRecords)]
    
    # Store the results in a summary table;
    result_table =  pd.DataFrame.from_dict(
                        allRecords,
                        orient='index',
                        columns=["QueriedRecordID", "Precision@K", "SelectedRecords", "AllRelevantRecords"]
                    )
    # Compute the filtered recall, which considers only queries with at least one relevant record in the training data;
    queries_with_relevant_records = result_table[result_table["AllRelevantRecords"] > 0]
    filtered_precision = np.mean(queries_with_relevant_records["SelectedRecords"] / numberOfPredictionsForRelevantRecords)

    return {
            "AveragePrecision" : totalPrecision / len(perQueryRecords),
            "AverageFilteredPrecision": filtered_precision,
            "perQueryResult" : result_table
            }   

#%%




@timeit
def prediction_dict_to_df(predictions: dict) -> pd.DataFrame:
    # Turn the prediction dict into a series of tuples;
    results = []
    for query_id, pred_list in predictions.items():
        for p in pred_list:
            results += [[query_id, p]]
    return pd.DataFrame(results, columns=["queried_record_id", "predicted_record_id"])

@timeit
def prediction_dict_to_kaggle_df(predictions: dict) -> pd.DataFrame:
    # Turn the prediction dict into a series of tuples;
    results = []
    for query_id, pred_list in predictions.items():
        results += [[query_id, " ".join(pred_list)]]
    return pd.DataFrame(results, columns=["queried_record_id", "predicted_record_id"])





def awesome_cossim_top(A, B, ntop, lower_bound=0,use_threads=True,n_jobs=14):
    # force A and B as a CSR matrix.
    # If they have already been CSR, there is no overhead
    A = A.tocsr()
    B = B.tocsr()
    M, _ = A.shape
    _, N = B.shape
    
    idx_dtype = np.int32
 
    nnz_max = M*ntop
 
    indptr = np.zeros(M+1, dtype=idx_dtype)
    indices = np.zeros(nnz_max, dtype=idx_dtype)
    data = np.zeros(nnz_max, dtype=A.dtype)
    ct_thread.sparse_dot_topn_threaded(
        M, N, np.asarray(A.indptr, dtype=idx_dtype),
        np.asarray(A.indices, dtype=idx_dtype),
        A.data,
        np.asarray(B.indptr, dtype=idx_dtype),
        np.asarray(B.indices, dtype=idx_dtype),
        B.data,
        ntop,
        lower_bound,
        indptr, indices, data, n_jobs)
    return csr_matrix((data,indices,indptr),shape=(M,N))

def make_matchdf(x,source,target):
    ''' Build dataframe for result return '''
    # CSR matrix -> COO matrix
    cx = x.tocoo()

    # COO matrix to list of tuples
    match_list = []
    for row,col,val in zip(cx.row, cx.col, cx.data):
        match_list.append((row,source[row], col, target[col], val))

    # List of tuples to dataframe
    colnames = ['Row Idx', 'Title', 'Candidate Idx', 'Candidate Title', 'Score']
    match_df = pd.DataFrame(match_list, columns=colnames)

    return match_df

#Function useful to divide the score matrix by the weight matrix
def sparse_divide_nonzero(a, b):
    inv_b = b.copy()
    inv_b.data = 1 / inv_b.data
    return a.multiply(inv_b)


def top_n_idx_sparse(matrix, n):
    '''Return index of top n values in each row of a sparse matrix'''
    top_n_idx = []
    for le, ri in zip(matrix.indptr[:-1], matrix.indptr[1:]):
        n_row_pick = min(n, ri - le)
        top_n_idx.append(matrix.indices[le + np.argpartition(matrix.data[le:ri], -n_row_pick)[-n_row_pick:]])
    return top_n_idx


In [3]:
#Loading dataset
train=pd.read_csv('../panama-papers-polimi/data/panama_train_expanded_2.csv')
test=pd.read_csv('../panama-papers-polimi/data/panama_test_expanded_2.csv')


training_file = "../panama-papers-polimi/data/entity-resolution_advanced-topics-training_data.csv"
train_or = read_file(training_file, set_record_id_as_index=False)
        
#X_train = train.drop(columns="linked_id")
y_train_or = train_or[["linked_id"]]
train.phone=train.phone.apply(str)
test.phone=test.phone.apply(str)

In [4]:
#Treat strings that are too short as missing values
train.name=train.name.apply(lambda x : '-1' if len(x)<=2 else x)
train.address=train.address.apply(lambda x : '-1' if len(x)<=2 else x)
train.email=train.email.apply(lambda x : '-1' if len(x)<=2 else x)
train.phone=train.phone.apply(lambda x : '-1' if len(x)<=2 else x)

test.name=test.name.apply(lambda x : '-1' if len(x)<=2 else x)
test.address=test.address.apply(lambda x : '-1' if len(x)<=2 else x)
test.email=test.email.apply(lambda x : '-1' if len(x)<=2 else x)
test.phone=test.phone.apply(lambda x : '-1' if len(x)<=2 else x)

train=train.fillna('-1')
test=test.fillna('-1')

train = pd.concat([train,y_train_or['linked_id']],axis=1)

In [5]:
###################
#Dataset piccolino
###################



#m1=(train.linked_id=='15005501')|(train.linked_id=='13013105')|(train.linked_id=='13013089')|(train.linked_id=='13013092')|(train.linked_id=='13013071')|(train.linked_id=='13013058')|(train.linked_id=='13013055')|(train.linked_id=='13013041')|(train.linked_id=='13013044')
#m2=(test.record_id=='15005501-TST-M')|(test.record_id=='13013105-T0-TST-CP')|(test.record_id=='13013089-TST-CP')|(test.record_id=='13013089-NV0-TST-CP')|(test.record_id=='13013089-T4-TST-CP')|(test.record_id=='13013092-NV0-TST-M')|(test.record_id=='13013092-TST-CP')|(test.record_id=='13013072-M1-TST-CP')|(test.record_id=='13013072-TST-CP')|(test.record_id=='10121215-M1')|(test.record_id=='10042968-T1') 



#train=train[m1]
#test=test[m2]

#train.reset_index(  drop=True, inplace=True)
#test.reset_index( drop=True, inplace=True)

In [6]:
#Create list from pd.Series
test_name_id=test.record_id.tolist()
test_ad_id=test.record_id.tolist()
test_em_id=test.record_id.tolist()
test_nu_id=test.record_id.tolist()


train_name_id=train.record_id.tolist()
train_ad_id=train.record_id.tolist()
train_em_id=train.record_id.tolist()
train_nu_id=train.record_id.tolist()


test_name_list=test.name.tolist()
test_address_list=test.address.tolist()
test_email_list=test.email.tolist()
test_number_list=test.phone.tolist()


train_name_list=train.name.tolist()
train_address_list=train.address.tolist()
train_email_list=train.email.tolist()
train_number_list=train.phone.tolist()


In [7]:
#Save indices of missing values to set them as zeros
test_na_series=pd.Series(test_name_list)
test_na_null=test_na_series[test_na_series=='-1'].index

test_ad_series=pd.Series(test_address_list)
test_ad_null=test_ad_series[test_ad_series=='-1'].index

test_em_series=pd.Series(test_email_list)
test_em_null=test_em_series[test_em_series=='-1'].index

test_nu_series=pd.Series(test_number_list)
test_nu_null=test_nu_series[test_nu_series=='-1'].index

train_na_series=pd.Series(train_name_list)
train_na_null=train_na_series[train_na_series=='-1'].index

train_ad_series=pd.Series(train_address_list)
train_ad_null=train_ad_series[train_ad_series=='-1'].index

train_em_series=pd.Series(train_email_list)
train_em_null=train_em_series[train_em_series=='-1'].index

train_nu_series=pd.Series(train_number_list)
train_nu_null=train_nu_series[train_nu_series=='-1'].index

In [8]:
#Create tfidf matrices
analyzer='char_wb'
n=3

ct_vect = CountVectorizer(analyzer=analyzer, ngram_range=(n-1, n))
vocab   = ct_vect.fit(test_name_list  + train_name_list).vocabulary_
tfidf_vect  = TfidfVectorizer(vocabulary=vocab, analyzer=analyzer, ngram_range=(n-1, n))
tf_idf_matrix_source_na=tfidf_vect.fit_transform(test_name_list)
tf_idf_matrix_target_na=tfidf_vect.fit_transform(train_name_list)


ct_vect_ad = CountVectorizer(analyzer=analyzer, ngram_range=(n-1, n))
vocab_ad   = ct_vect_ad.fit(test_address_list  + train_address_list).vocabulary_
tfidf_vect_ad  = TfidfVectorizer(vocabulary=vocab_ad, analyzer=analyzer, ngram_range=(n-1, n))
tf_idf_matrix_source_ad=tfidf_vect_ad.fit_transform(test_address_list)
tf_idf_matrix_target_ad=tfidf_vect_ad.fit_transform(train_address_list)

ct_vect_em = CountVectorizer(analyzer=analyzer, ngram_range=(n-1, n))
vocab_em   = ct_vect_em.fit(test_email_list  + train_email_list).vocabulary_
tfidf_vect_em  = TfidfVectorizer(vocabulary=vocab_em, analyzer=analyzer, ngram_range=(n-1, n))
tf_idf_matrix_source_em=tfidf_vect_em.fit_transform(test_email_list)
tf_idf_matrix_target_em=tfidf_vect_em.fit_transform(train_email_list)


ct_vect_nu = CountVectorizer(analyzer=analyzer, ngram_range=(n-1, n))
vocab_nu   = ct_vect_nu.fit(test_number_list  + train_number_list).vocabulary_
tfidf_vect_nu  = TfidfVectorizer(vocabulary=vocab_nu, analyzer=analyzer, ngram_range=(n-1, n))
tf_idf_matrix_source_nu=tfidf_vect_nu.fit_transform(test_number_list)
tf_idf_matrix_target_nu=tfidf_vect_nu.fit_transform(train_number_list)


In [9]:
#Set tfidf vectors of missing values in the test set as zeros cause their similarity scores are useless
for a in test_na_null:
    tf_idf_matrix_source_na.data[tf_idf_matrix_source_na.indptr[a]:tf_idf_matrix_source_na.indptr[a+1]]=0

#for b in train_na_null:
#    tf_idf_matrix_target_na.data[tf_idf_matrix_target_na.indptr[b]:tf_idf_matrix_target_na.indptr[b+1]]=0
    
    
for c in test_ad_null:
    tf_idf_matrix_source_ad.data[tf_idf_matrix_source_ad.indptr[c]:tf_idf_matrix_source_ad.indptr[c+1]]=0

#for d in train_ad_null:
#    tf_idf_matrix_target_ad.data[tf_idf_matrix_target_ad.indptr[d]:tf_idf_matrix_target_ad.indptr[d+1]]=0
    
    
for e in test_em_null:
    tf_idf_matrix_source_em.data[tf_idf_matrix_source_em.indptr[e]:tf_idf_matrix_source_em.indptr[e+1]]=0

#for f in train_em_null:
#    tf_idf_matrix_target_em.data[tf_idf_matrix_target_em.indptr[f]:tf_idf_matrix_target_em.indptr[f+1]]=0    

    

for g in test_nu_null:
    tf_idf_matrix_source_nu.data[tf_idf_matrix_source_nu.indptr[g]:tf_idf_matrix_source_nu.indptr[g+1]]=0

#for h in train_nu_null:
#    tf_idf_matrix_target_nu.data[tf_idf_matrix_target_nu.indptr[h]:tf_idf_matrix_target_nu.indptr[h+1]]=0
    

In [10]:
t1 = time.time()
matches_em = awesome_cossim_top(tf_idf_matrix_source_em, tf_idf_matrix_target_em.transpose(),691440, 0)
t = time.time()-t1
print("SELFTIMED:", t)


SELFTIMED: 703.3167402744293


In [11]:
t1 = time.time()
matches_ad = awesome_cossim_top(tf_idf_matrix_source_ad, tf_idf_matrix_target_ad.transpose(),691440, 0)
t = time.time()-t1
print("SELFTIMED:", t)

SELFTIMED: 1215.2100429534912


In [12]:
t1 = time.time()
matches_nu = awesome_cossim_top(tf_idf_matrix_source_nu, tf_idf_matrix_target_nu.transpose(),691440, 0)
t = time.time()-t1
print("SELFTIMED:", t)

SELFTIMED: 750.881795167923


In [13]:
t1 = time.time()
matches_na = awesome_cossim_top(tf_idf_matrix_source_na, tf_idf_matrix_target_na.transpose(),691440, 0)
t = time.time()-t1
print("SELFTIMED:", t)

SELFTIMED: 2227.880414247513


In [15]:
#Let's create also csc format to set columns efficiently to zero 
name_csc=matches_na.tocsc()
ad_csc=matches_ad.tocsc()
em_csc=matches_em.tocsc()
nu_csc=matches_nu.tocsc()

In [16]:
#Now it's time to set similarity scores with nan train features as zeros
for i in train_na_null:
    name_csc.data[name_csc.indptr[i]:name_csc.indptr[i+1]]=0

for j in train_ad_null:
    ad_csc.data[ad_csc.indptr[j]:ad_csc.indptr[j+1]]=0

for k in train_em_null:
    em_csc.data[em_csc.indptr[k]:em_csc.indptr[k+1]]=0
    
for l in train_nu_null:
    nu_csc.data[nu_csc.indptr[l]:nu_csc.indptr[l+1]]=0

In [17]:
#Let's switch to coo format to sum the csc matrics
name_coo=name_csc.tocoo()
ad_coo=ad_csc.tocoo()
em_coo=em_csc.tocoo()
nu_coo=nu_csc.tocoo()

In [18]:
#Let's sum the matrices of similarity scores
mat_fin=name_coo+ad_coo+em_coo+nu_coo

In [ ]:
#Creation of masks just for instances not excluded from top-n

#na_w_mask = np.array(name_csc[name_csc.nonzero()] <=1)[0]
#ad_w_mask = np.array(ad_csc[ad_csc.nonzero()] <=1)[0]
#em_w_mask = np.array(em_csc[em_csc.nonzero()] <=1)[0]
#nu_w_mask = np.array(nu_csc[nu_csc.nonzero()] <=1)[0]

In [ ]:
#rows_na = name_csc.nonzero()[0][na_w_mask]
#cols_na = name_csc.nonzero()[1][na_w_mask]

#rows_ad = ad_csc.nonzero()[0][ad_w_mask]
#cols_ad = ad_csc.nonzero()[1][ad_w_mask]

#rows_em = em_csc.nonzero()[0][em_w_mask]
#cols_em = em_csc.nonzero()[1][em_w_mask]

#rows_nu = nu_csc.nonzero()[0][nu_w_mask]
#cols_nu = nu_csc.nonzero()[1][nu_w_mask]

In [40]:
weights_na=name_csc.copy()
weights_ad=ad_csc.copy()
weights_em=em_csc.copy()
weights_nu=nu_csc.copy()

#Uniform weights

#weights_na[rows_na,cols_na]=1
#weights_ad[rows_ad,cols_ad]=1
#weights_em[rows_em,cols_em]=1
#weights_nu[rows_nu,cols_nu]=1

In [20]:
#Save indices of missing values to set them as zeros
test_na_series=pd.Series(test_name_list)
test_na_not_null=test_na_series[test_na_series!='-1'].index

test_ad_series=pd.Series(test_address_list)
test_ad_not_null=test_ad_series[test_ad_series!='-1'].index

test_em_series=pd.Series(test_email_list)
test_em_not_null=test_em_series[test_em_series!='-1'].index

test_nu_series=pd.Series(test_number_list)
test_nu_not_null=test_nu_series[test_nu_series!='-1'].index

train_na_series=pd.Series(train_name_list)
train_na_not_null=train_na_series[train_na_series!='-1'].index

train_ad_series=pd.Series(train_address_list)
train_ad_not_null=train_ad_series[train_ad_series!='-1'].index

train_em_series=pd.Series(train_email_list)
train_em_not_null=train_em_series[train_em_series!='-1'].index

train_nu_series=pd.Series(train_number_list)
train_nu_not_null=train_nu_series[train_nu_series!='-1'].index

In [ ]:
#Creation of correct weights matrices

start=time.time()
for j in train_ad_not_null:
    weights_ad.data[weights_ad.indptr[j]:weights_ad.indptr[j+1]]=1
print(time.time()-start)

/home/users/michele.bertoldi/anaconda3/lib/python3.6/site-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [28]:
for j in train_em_not_null:
    weights_em.data[weights_em.indptr[j]:weights_em.indptr[j+1]]=1
    
for j in train_na_not_null:
    weights_na.data[weights_na.indptr[j]:weights_na.indptr[j+1]]=1   

for j in train_nu_not_null:
    weights_nu.data[weights_nu.indptr[j]:weights_nu.indptr[j+1]]=1

In [29]:
#---> COO format to execute sum of the weights matrices
weights_na_coo=weights_na.tocoo()
weights_ad_coo=weights_ad.tocoo()
weights_em_coo=weights_em.tocoo()
weights_nu_coo=weights_nu.tocoo()

In [30]:
#Matrix of weights
weight_tot=weights_na_coo+weights_ad_coo+weights_em_coo+weights_nu_coo

In [31]:
#Let's finally perform the division
M3 = sparse_divide_nonzero(mat_fin, weight_tot)

In [33]:
train_linked_id_list=train.linked_id.tolist()

In [34]:
match_df={}
list_top=top_n_idx_sparse(M3,10)
i=0
for row_i in zip(test['record_id'],test['name']):
    match_df[row_i[0]]=[train_linked_id_list[j] for j in list_top[i]]
    i+=1

In [ ]:
list_top

In [36]:
match_df

['12075168',
 '12096573',
 '12096573',
 '10080397',
 '10145923',
 '12104777',
 '12133698',
 '12133698',
 '10201125',
 '12133698']

In [37]:
csv_name = 'tfidf_independent_top10'
csv_name += datetime.now().strftime('%b%d_%H-%M-%S')+'.csv'
#pred_df.to_csv("../panama-oracle-f-2/"+csv_name, index=False)

pred_df_kaggle = prediction_dict_to_kaggle_df(match_df)

pred_df_kaggle.to_csv("../panama-papers-polimi/"+csv_name, index=False)
